# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5db8242730>
├── 'a' --> tensor([[0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133]])
└── 'x' --> <FastTreeValue 0x7f5db82422e0>
    └── 'c' --> tensor([[ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-02],
                        [ 3.2517e-01, -2.5166e-04,  2.5517e-01,  1.5165e+00],
                        [ 9.0354e-01, -1.2548e-01,  5.7880e-01, -1.1625e+00]])

In [4]:
t.a

tensor([[0.6953, 1.0526, 0.2227],
        [0.1908, 0.2277, 0.0133]])

In [5]:
%timeit t.a

77.5 ns ± 0.584 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5db8242730>
├── 'a' --> tensor([[-1.0070,  0.7708,  1.2613],
│                   [-0.0665,  0.6317, -0.8081]])
└── 'x' --> <FastTreeValue 0x7f5db82422e0>
    └── 'c' --> tensor([[ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-02],
                        [ 3.2517e-01, -2.5166e-04,  2.5517e-01,  1.5165e+00],
                        [ 9.0354e-01, -1.2548e-01,  5.7880e-01, -1.1625e+00]])

In [7]:
%timeit t.a = new_value

80.1 ns ± 1.6 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133]]),
    x: Batch(
           c: tensor([[ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-02],
                      [ 3.2517e-01, -2.5166e-04,  2.5517e-01,  1.5165e+00],
                      [ 9.0354e-01, -1.2548e-01,  5.7880e-01, -1.1625e+00]]),
       ),
)

In [10]:
b.a

tensor([[0.6953, 1.0526, 0.2227],
        [0.1908, 0.2277, 0.0133]])

In [11]:
%timeit b.a

75.4 ns ± 0.702 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4856, -0.1999, -0.7615],
               [ 0.0727, -0.2552, -1.6941]]),
    x: Batch(
           c: tensor([[ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-02],
                      [ 3.2517e-01, -2.5166e-04,  2.5517e-01,  1.5165e+00],
                      [ 9.0354e-01, -1.2548e-01,  5.7880e-01, -1.1625e+00]]),
       ),
)

In [13]:
%timeit b.a = new_value

661 ns ± 6.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.1 µs ± 9.18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.8 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

297 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

282 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5cf4689dc0>
├── 'a' --> tensor([[[0.6953, 1.0526, 0.2227],
│                    [0.1908, 0.2277, 0.0133]],
│           
│                   [[0.6953, 1.0526, 0.2227],
│                    [0.1908, 0.2277, 0.0133]],
│           
│                   [[0.6953, 1.0526, 0.2227],
│                    [0.1908, 0.2277, 0.0133]],
│           
│                   [[0.6953, 1.0526, 0.2227],
│                    [0.1908, 0.2277, 0.0133]],
│           
│                   [[0.6953, 1.0526, 0.2227],
│                    [0.1908, 0.2277, 0.0133]],
│           
│                   [[0.6953, 1.0526, 0.2227],
│                    [0.1908, 0.2277, 0.0133]],
│           
│                   [[0.6953, 1.0526, 0.2227],
│                    [0.1908, 0.2277, 0.0133]],
│           
│                   [[0.6953, 1.0526, 0.2227],
│                    [0.1908, 0.2277, 0.0133]]])
└── 'x' --> <FastTreeValue 0x7f5cf4689e80>
    └── 'c' --> tensor([[[ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-

In [21]:
%timeit t_stack(trees)

52.1 µs ± 548 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5ceca547c0>
├── 'a' --> tensor([[0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133],
│                   [0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133],
│                   [0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133],
│                   [0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133],
│                   [0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133],
│                   [0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133],
│                   [0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133],
│                   [0.6953, 1.0526, 0.2227],
│                   [0.1908, 0.2277, 0.0133]])
└── 'x' --> <FastTreeValue 0x7f5dbc257160>
    └── 'c' --> tensor([[ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-02],
                        [ 3.2517e-01, -2.5166e-04,  2.5517e-01,  1.5165e+00],
                        [ 9.0354e

In [23]:
%timeit t_cat(trees)

48.4 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

111 µs ± 4.53 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[0.6953, 1.0526, 0.2227],
                [0.1908, 0.2277, 0.0133]],
       
               [[0.6953, 1.0526, 0.2227],
                [0.1908, 0.2277, 0.0133]],
       
               [[0.6953, 1.0526, 0.2227],
                [0.1908, 0.2277, 0.0133]],
       
               [[0.6953, 1.0526, 0.2227],
                [0.1908, 0.2277, 0.0133]],
       
               [[0.6953, 1.0526, 0.2227],
                [0.1908, 0.2277, 0.0133]],
       
               [[0.6953, 1.0526, 0.2227],
                [0.1908, 0.2277, 0.0133]],
       
               [[0.6953, 1.0526, 0.2227],
                [0.1908, 0.2277, 0.0133]],
       
               [[0.6953, 1.0526, 0.2227],
                [0.1908, 0.2277, 0.0133]]]),
    x: Batch(
           c: tensor([[[ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-02],
                       [ 3.2517e-01, -2.5166e-04,  2.5517e-01,  1.5165e+00],
                       [ 9.0354e-01, -1.2548e-01,  5.7880e-01, -1.1625e+00]],
         

In [26]:
%timeit Batch.stack(batches)

118 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133],
               [0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133],
               [0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133],
               [0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133],
               [0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133],
               [0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133],
               [0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133],
               [0.6953, 1.0526, 0.2227],
               [0.1908, 0.2277, 0.0133]]),
    x: Batch(
           c: tensor([[ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-02],
                      [ 3.2517e-01, -2.5166e-04,  2.5517e-01,  1.5165e+00],
                      [ 9.0354e-01, -1.2548e-01,  5.7880e-01, -1.1625e+00],
                      [ 1.0578e+00,  3.0921e-01, -2.1333e+00, -8.6080e-02],
                 

In [28]:
%timeit Batch.cat(batches)

211 µs ± 6.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

703 µs ± 60.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
